In [ ]:
#Import data

In [79]:
from __future__ import print_function  # Python 2 and 3
import pandas as pd
import scipy.stats
import numpy as np
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn import linear_model
import xgboost as xgb
from sklearn.svm import SVR
#from scipy import stats
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline
plt.style.use('ggplot')
import statsmodels.api as sm 

# Read in Training Set

In [80]:
df = pd.read_csv('post_process_train.csv', index_col = 0)
#df = df[['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape',
        # 'LandContour', 'Utilities', 'LotConfig', 'SalePrice']].dropna()

In [81]:
df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', 'X1stFlrSF', 'X2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDri

# Read in Test Data

In [82]:
df_test = pd.read_csv('post_process_test.csv', index_col = 0).dropna()
#df_test = df_test[['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape',
        # 'LandContour', 'Utilities', 'LotConfig']].dropna()

In [83]:
df_test.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', 'X1stFlrSF', 'X2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDri

# Process Data

In [84]:
#process training data
categorical_columns = ['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour',
                       'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
                      'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 
                      'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd',
                       'MasVnrType',
                       'Foundation', 'BsmtFinType1', 'BsmtFinType2',
                       'Heating', 'CentralAir', 'Electrical', 'Functional', 
                       'GarageType', 'GarageFinish', 'PavedDrive', 'Fence',
                       'MiscFeature', 'SaleType', 'SaleCondition']
#numerical_columns = ['LotFrontage', 'LotArea', 'SalePrice']
column_trans = make_column_transformer((OneHotEncoder(handle_unknown='ignore',sparse = False),categorical_columns))
x = column_trans.fit_transform(df)

In [85]:
#process test data
categorical_columns = ['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour',
                       'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
                      'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 
                      'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd',
                       'MasVnrType',
                       'Foundation', 'BsmtFinType1', 'BsmtFinType2',
                       'Heating', 'CentralAir', 'Electrical', 'Functional', 
                       'GarageType', 'GarageFinish', 'PavedDrive', 'Fence',
                       'MiscFeature', 'SaleType', 'SaleCondition']
#numerical_columns = ['LotFrontage', 'LotArea', 'SalePrice']
column_trans = make_column_transformer((OneHotEncoder(handle_unknown='ignore', sparse = False),categorical_columns))
test_x = column_trans.fit_transform(df_test)

# Box Cox Transformation

In [86]:
df["SalePrice"] = scipy.stats.boxcox(df["SalePrice"], .25)
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1,1,60,RL,65,8450,Pave,None,Reg,Lvl,AllPub,...,0,0,None,None,0,2,2008,WD,Normal,81.474488
2,2,20,RL,80,9600,Pave,None,Reg,Lvl,AllPub,...,0,0,None,None,0,5,2007,WD,Normal,78.561799
3,3,60,RL,68,11250,Pave,None,IR1,Lvl,AllPub,...,0,0,None,None,0,9,2008,WD,Normal,82.971983
4,4,70,RL,60,9550,Pave,None,IR1,Lvl,AllPub,...,0,0,None,None,0,2,2006,WD,Abnorml,73.373457
5,5,60,RL,84,14260,Pave,None,IR1,Lvl,AllPub,...,0,0,None,None,0,12,2008,WD,Normal,85.442719


#AIC & BIC

In [77]:
X_add_const = sm.add_constant(x)
ols = sm.OLS(df['SalePrice'], X_add_const)
ans = ols.fit()
print(ans.summary())

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.827
Model:                            OLS   Adj. R-squared:                  0.801
Method:                 Least Squares   F-statistic:                     31.83
Date:                Fri, 14 Aug 2020   Prob (F-statistic):               0.00
Time:                        15:06:35   Log-Likelihood:                -3860.4
No. Observations:                1460   AIC:                             8105.
Df Residuals:                    1268   BIC:                             9120.
Df Model:                         191                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          8.8647      0.498     17.790      0.0

#Multiple Linear Regression

In [ ]:
ols = linear_model.LinearRegression()

# Random Forest

In [87]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 100)
rf.fit(x,df['SalePrice'])

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [88]:
rf.score(x, df['SalePrice'])

0.9572226497801489

In [89]:
rf.predict(test_x)

ValueError: Number of features of the model must match the input. Model n_features is 227 and input n_features is 211 

# Beginning of XGBoost

In [90]:
mod = xgb.XGBRegressor(
    gamma=1,                 
    learning_rate=0.01,
    max_depth=3,
    n_estimators=10000,                                                                    
    subsample=0.8,
    random_state=34
) 

mod.fit(x,df['SalePrice'])

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=1, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=10000, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=34, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=0.8,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [92]:
mod.score(x,df['SalePrice'])

0.9541765577928563

In [93]:
mod.predict(test_x)

ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128', 'f129', 'f130', 'f131', 'f132', 'f133', '

# SVR

In [94]:
svr = SVR() #epsilon = 1e-4, gamma=1
svr.fit(x,df["SalePrice"])
print(svr.score(x,df["SalePrice"]))

0.6935778302692172


# Undo Box Cox Transformation

In [53]:
df["SalePrice"] = scipy.special.inv_boxcox(df["SalePrice"], 0.25)
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1,1,60,RL,65,8450,Pave,None,Reg,Lvl,AllPub,...,0,0,None,None,0,2,2008,WD,Normal,208500.0
2,2,20,RL,80,9600,Pave,None,Reg,Lvl,AllPub,...,0,0,None,None,0,5,2007,WD,Normal,181500.0
3,3,60,RL,68,11250,Pave,None,IR1,Lvl,AllPub,...,0,0,None,None,0,9,2008,WD,Normal,223500.0
4,4,70,RL,60,9550,Pave,None,IR1,Lvl,AllPub,...,0,0,None,None,0,2,2006,WD,Abnorml,140000.0
5,5,60,RL,84,14260,Pave,None,IR1,Lvl,AllPub,...,0,0,None,None,0,12,2008,WD,Normal,250000.0
